<a href="https://colab.research.google.com/github/HSUAN221/colab/blob/main/pdf_adventurer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install faiss-cpu
!pip install langchain
!pip install openai
!pip install PyPDF2

In [22]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate, FAISS

In [3]:
# Get your API keys from openai, you will need to create an account. 
# Here is the link to get the keys: https://platform.openai.com/account/billing/overview
import os
os.environ["OPENAI_API_KEY"] = ""

In [38]:
# connect your Google Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"

Mounted at /content/gdrive


In [56]:
# location of the pdf file/files. 
reader = PdfReader('/content/gdrive/MyDrive/pdfs/Hanser_ED2_Ch02.pdf')
reader

In [57]:
# read data from the file and put them into a variable called raw_text
raw_text = ''
for i, page in enumerate(reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text

In [58]:
# We need to split the text that we read into smaller chunks so that during information retreival we don't hit the token size limits. 

text_splitter = CharacterTextSplitter(        
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [59]:
# Download embeddings from OpenAI
embeddings = OpenAIEmbeddings()

In [60]:
docsearch = FAISS.from_texts(texts, embeddings)

In [61]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [84]:
chain = load_qa_chain(OpenAI(model_name="text-davinci-003"), chain_type="stuff")

In [85]:
query = "What is the Molecular structure?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' The molecular structure of a polymer involves atoms connected by covalent bonding forces and different chain conformations.'

In [72]:
query = "What is the Effects of Shear Rate?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' Shear rate increases, the polymer chains start to disentangle and align along the direction of flow. The alignment leads to less molecular interaction and the viscosity continues to drop, causing the phenomenon of shear thinning behavior. When the shear rate gets extremely high, the viscosity reaches a minimum (η\uf0a5) theoretically, though this is usually difficult to measure due to the acute degradation of the material under such high shear rate and stress.'

In [73]:
query = "What is the Effects of Temperature?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' Raising the processing temperature is a common processing procedure often used to increase the flowability of polymer melts. The viscosity of polymeric fluids is not only sensitive to the shear rate, but also changes with temperature. The curve of viscosity moves toward the bottom left side of the graph as temperature rises. This is because the polymer chains have larger free volume and mobility that are much easier to overcome the interaction energy of molecules.'

In [75]:
query = "What is Effects of Pressure?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' Pressure can affect the viscosity of polymeric fluids, but the effect may not be very significant in a common injection molding filling process. However, for high-speed injection molding of thin-wall parts that uses pressure over 100 MPa, the viscosity increases with pressure starts to become significant. The pressure effect can also be described by free volume theory that the free volume decreases with increasing pressure.'